### LangChain 고급 검색 기법 (Advanced Retrieval)

기본적인 유사도 검색만으로는 부족할 때가 많음. 그래서 나온 게 바로 '고급 검색 기법'들임.

이 노트북에서는 다음과 같은 내용을 다룰 예정:
1. **환경 설정**: 기본 세팅.
2. **샘플 데이터 및 ChromaDB 생성**: 실제 코드를 돌려보기 위한 필수 과정!
3. **기본 검색**: 비교를 위한 기준선.
4. **쿼리 확장**: 질문을 다양하게 변형해서 검색.
5. **재순위화 (Re-rank)**: 검색 결과를 한 번 더 똑똑하게 정렬.
6. **맥락적 압축 (Contextual Compression)**: 필요한 정보만 쏙쏙 뽑아내기.
7. **RAG 파이프라인으로 답변 생성**: 최종적으로 LLM이 답변하는 과정.

## 1. 환경 설정

`(1) 기본 라이브러리`

In [1]:
import os
from glob import glob

from pprint import pprint
import json

import numpy as np
import pandas as pd

# LangChain 관련 라이브러리들
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, BaseOutputParser
from langchain_core.runnables import RunnablePassthrough

# 문서 로더 및 분할기
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 고급 검색 관련
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker, LLMListwiseRerank, LLMChainFilter, LLMChainExtractor, EmbeddingsFilter, DocumentCompressorPipeline
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.document_transformers import EmbeddingsRedundantFilter

from typing import List

In [2]:
from dotenv import load_dotenv
load_dotenv() 

True

### 2. 샘플 데이터 및 ChromaDB 생성

**과정:**
1.  **샘플 텍스트 파일 생성**: `data` 폴더 만들고 그 안에 `tesla.txt`, `rivian.txt`, `general_tech.txt` 파일 생성.
2.  **문서 로드 (Load)**: `DirectoryLoader`로 `data` 폴더 안의 모든 `.txt` 파일 읽어오기.
3.  **문서 분할 (Split)**: 긴 문서를 적절한 크기로 쪼개기 (`RecursiveCharacterTextSplitter`). LLM이 한 번에 처리할 수 있는 양이 정해져 있기 때문임.
4.  **임베딩 (Embed)**: 쪼갠 문서 조각들을 숫자로 된 벡터로 변환 (`HuggingFaceEmbeddings`). 그래야 컴퓨터가 의미를 이해하고 유사도를 계산할 수 있음.
5.  **ChromaDB에 저장 (Store)**: 변환된 벡터들을 ChromaDB에 저장하고, 나중에 다시 불러올 수 있도록 디스크에도 저장 (`persist_directory`).

In [3]:
# 샘플 데이터 저장할 디렉토리 생성
if not os.path.exists("sample_data"):
    os.makedirs("sample_data")

# 샘플 텍스트 파일 내용
tesla_content = """
Tesla, Inc.는 미국의 전기 자동차 및 청정 에너지 회사입니다. 
본사는 텍사스 오스틴에 있습니다. Tesla는 전기 자동차, 태양광 패널, 배터리 에너지 저장 장치를 설계하고 제조합니다.
일론 머스크 (Elon Musk)는 Tesla의 CEO이자 제품 설계자입니다. 
그는 Tesla의 장기적인 전략적 방향과 모든 제품 개발을 이끌고 있습니다.
로빈 덴홈 (Robyn Denholm)은 Tesla 이사회 의장입니다.
Tesla의 주요 차량 모델에는 Model S, Model 3, Model X, Model Y, 그리고 Cybertruck이 있습니다.
"""

rivian_content = """
Rivian Automotive, Inc.는 미국의 전기 자동차 제조업체이자 자동차 기술 회사입니다. 
본사는 캘리포니아 어바인에 있습니다. Rivian은 전기 SUV와 픽업 트럭을 생산합니다.
Rivian의 첫 번째 차량은 2021년에 공개된 R1T 전기 픽업 트럭입니다.
R1S라는 SUV 모델도 있습니다. Rivian은 배송용 전기 밴도 개발하여 아마존에 공급하고 있습니다.
Rivian의 성장 동력은 혁신적인 전기차 기술과 강력한 시장 수요입니다.
회사는 2009년에 설립되었으며, 초기에는 다른 이름으로 시작했으나 2011년에 Rivian으로 변경했습니다.
"""

general_tech_content = """
인공지능(AI)은 빠르게 발전하고 있는 기술 분야입니다.
머신러닝과 딥러닝은 AI의 핵심 구성 요소입니다.
최근에는 대규모 언어 모델(LLM)이 주목받고 있으며, 다양한 서비스에 활용되고 있습니다.
"""

with open("sample_data/tesla.txt", "w", encoding="utf-8") as f:
    f.write(tesla_content)

with open("sample_data/rivian.txt", "w", encoding="utf-8") as f:
    f.write(rivian_content)

with open("sample_data/general_tech.txt", "w", encoding="utf-8") as f:
    f.write(general_tech_content)

print("샘플 파일 생성 완료: sample_data/tesla.txt, sample_data/rivian.txt, sample_data/general_tech.txt")

샘플 파일 생성 완료: sample_data/tesla.txt, sample_data/rivian.txt, sample_data/general_tech.txt


In [4]:
# 1. 문서 로드
loader = DirectoryLoader('./sample_data/', glob="*.txt", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
documents = loader.load()
print(f"총 {len(documents)}개의 문서 로드 완료.")

# 2. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(f"총 {len(docs)}개의 문서 조각으로 분할 완료.")

# 3. 임베딩 모델 설정
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3") # 한국어 지원 및 성능 좋은 모델
print("임베딩 모델 로드 완료: BAAI/bge-m3")

# 4. ChromaDB에 저장 (및 디스크에 영구 저장)
db_directory = "./chroma_db_advanced"
if os.path.exists(db_directory):
    # 기존 디렉토리가 있다면 삭제 (새로 만들기 위해)
    import shutil
    shutil.rmtree(db_directory)
    print(f"기존 ChromaDB 디렉토리 '{db_directory}' 삭제 완료.")

chroma_db = Chroma.from_documents(
    documents=docs, 
    embedding=embeddings_model, 
    collection_name="hf_bge_m3_advanced", # 컬렉션 이름 지정
    persist_directory=db_directory
)
print(f"ChromaDB 생성 및 저장 완료. 저장 경로: {db_directory}")

총 3개의 문서 로드 완료.
총 3개의 문서 조각으로 분할 완료.
임베딩 모델 로드 완료: BAAI/bge-m3
기존 ChromaDB 디렉토리 './chroma_db_advanced' 삭제 완료.
ChromaDB 생성 및 저장 완료. 저장 경로: ./chroma_db_advanced


### 3. 벡터저장소 로드 및 기본 검색

고급 검색 기법들의 성능을 비교하기 위한 '기준점'.
`as_retriever()`는 ChromaDB를 LangChain에서 쓸 수 있는 검색기(Retriever) 형태로 만들어줌.
`search_kwargs={"k": 2}`는 "가장 유사한 문서 2개 찾아줘"라는 뜻임.

In [5]:
chroma_db_loaded = Chroma(
    embedding_function=embeddings_model,
    collection_name="hf_bge_m3_advanced", # 생성 시 사용한 컬렉션 이름과 동일해야 함
    persist_directory="./chroma_db_advanced", # 생성 시 사용한 경로와 동일해야 함
)
print("ChromaDB 로드 완료.")

ChromaDB 로드 완료.


C:\Users\USER\AppData\Local\Temp\ipykernel_26408\4051912356.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db_loaded = Chroma(


In [6]:
# 기본 retriever 초기화
chroma_k_retriever = chroma_db_loaded.as_retriever(
    search_kwargs={"k": 2} # 상위 2개 문서 검색
)

query = "리비안의 성장 동력은 무엇인가요?"
retrieved_docs_base = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("기본 검색 결과:")
for i, doc in enumerate(retrieved_docs_base):
    print(f"문서 {i+1}:")
    print(f"- 내용: {doc.page_content}")
    print(f"- 출처: {doc.metadata['source']}")
    print("-"*50)

쿼리: 리비안의 성장 동력은 무엇인가요?
기본 검색 결과:
문서 1:
- 내용: Rivian Automotive, Inc.는 미국의 전기 자동차 제조업체이자 자동차 기술 회사입니다. 
본사는 캘리포니아 어바인에 있습니다. Rivian은 전기 SUV와 픽업 트럭을 생산합니다.
Rivian의 첫 번째 차량은 2021년에 공개된 R1T 전기 픽업 트럭입니다.
R1S라는 SUV 모델도 있습니다. Rivian은 배송용 전기 밴도 개발하여 아마존에 공급하고 있습니다.
Rivian의 성장 동력은 혁신적인 전기차 기술과 강력한 시장 수요입니다.
회사는 2009년에 설립되었으며, 초기에는 다른 이름으로 시작했으나 2011년에 Rivian으로 변경했습니다.
- 출처: sample_data\rivian.txt
--------------------------------------------------
문서 2:
- 내용: 인공지능(AI)은 빠르게 발전하고 있는 기술 분야입니다.
머신러닝과 딥러닝은 AI의 핵심 구성 요소입니다.
최근에는 대규모 언어 모델(LLM)이 주목받고 있으며, 다양한 서비스에 활용되고 있습니다.
- 출처: sample_data\general_tech.txt
--------------------------------------------------


## 4. 고급 검색기법

### 4-1. 쿼리(Query) 확장

- 사용자가 입력한 질문 하나만 가지고 검색하면, 표현이 조금만 달라도 중요한 문서를 놓칠 수 있음.
- 그래서 원래 질문을 바탕으로 여러 개의 유사하거나 관련된 질문을 생성해서 검색하는 전략임.
- 마치 그물코를 여러 개 던져서 물고기를 잡는 것과 비슷함.

`(1) Multi Query Retriever`

**원리**: LLM을 사용해서 원래 질문을 다양한 관점에서 재해석한 여러 개의 새로운 질문(쿼리)을 생성함. 그리고 이 모든 쿼리로 각각 검색을 수행한 다음, 결과를 합쳐서 중복을 제거하고 반환함.

**장점**:
-   **향상된 재현율(Recall)**: 다양한 표현의 질문으로 검색하기 때문에, 원래 질문만으로는 찾기 어려웠던 관련 문서들을 찾아낼 가능성이 높아짐.
-   **질문 의도 파악**: 사용자의 모호하거나 복잡한 질문 의도를 여러 각도에서 해석해볼 수 있음.

**단점**:
-   **LLM 호출 비용 및 시간**: 여러 개의 쿼리를 생성하기 위해 LLM을 호출하므로 추가 비용과 시간이 발생함.
-   **불필요한 검색 증가**: 생성된 쿼리가 원래 의도와 동떨어지거나 너무 광범위하면, 관련 없는 문서를 많이 가져올 수 있음 (정확도 저하 가능성).
-   **결과 통합의 어려움**: 여러 쿼리에서 나온 결과를 어떻게 효과적으로 통합하고 우선순위를 매길지가 중요함.

- **MultiQueryRetriever 기본 사용**
  LangChain에 내장된 `MultiQueryRetriever.from_llm()`을 사용하면 기본 프롬프트로 쉽게 멀티 쿼리를 생성할 수 있음.

In [ ]:
llm_multi_query = ChatOpenAI(
    model='gpt-4o-mini', 
    temperature=0,
)

multi_query_retriever_default = MultiQueryRetriever.from_llm(
    retriever=chroma_k_retriever, # 위에서 정의한 기본 검색기 (k=2)
    llm=llm_multi_query
)

query_multi = "리비안의 성장 동력은 무엇인가요?"
#랭스미스 입력 확인 예: "1. 리비안의 성장 요인은 무엇이라고 할 수 있나요?", "2. 리비안의 발전을 이끄는 핵심적인 힘은 무엇인가요?", "리비안이 성장하는 데 기여하는 주요 요소는 무엇인가요?"

retrieved_docs_multi_default = multi_query_retriever_default.invoke(query_multi)

print(f"[MultiQueryRetriever 기본 사용] 쿼리: {query_multi}")
print("검색 결과:")
unique_docs_multi_default = {doc.page_content: doc for doc in retrieved_docs_multi_default} # 내용 기반 중복 제거
for i, doc in enumerate(unique_docs_multi_default.values()):
    print(f"문서 {i+1}:")
    print(f"- 내용: {doc.page_content}")
    print(f"- 출처: {doc.metadata['source']}")
    print("-"*50)

[MultiQueryRetriever 기본 사용] 쿼리: 리비안의 성장 동력은 무엇인가요?
검색 결과:
문서 1:
- 내용: Rivian Automotive, Inc.는 미국의 전기 자동차 제조업체이자 자동차 기술 회사입니다. 
본사는 캘리포니아 어바인에 있습니다. Rivian은 전기 SUV와 픽업 트럭을 생산합니다.
Rivian의 첫 번째 차량은 2021년에 공개된 R1T 전기 픽업 트럭입니다.
R1S라는 SUV 모델도 있습니다. Rivian은 배송용 전기 밴도 개발하여 아마존에 공급하고 있습니다.
Rivian의 성장 동력은 혁신적인 전기차 기술과 강력한 시장 수요입니다.
회사는 2009년에 설립되었으며, 초기에는 다른 이름으로 시작했으나 2011년에 Rivian으로 변경했습니다.
- 출처: sample_data\rivian.txt
--------------------------------------------------
문서 2:
- 내용: 인공지능(AI)은 빠르게 발전하고 있는 기술 분야입니다.
머신러닝과 딥러닝은 AI의 핵심 구성 요소입니다.
최근에는 대규모 언어 모델(LLM)이 주목받고 있으며, 다양한 서비스에 활용되고 있습니다.
- 출처: sample_data\general_tech.txt
--------------------------------------------------


**Custom Prompt 활용한 Multi Query**
  - 쿼리 생성 방식을 더 세밀하게 제어하고 싶다면, 직접 프롬프트를 작성해서 사용할 수 있음.
  - 이를 통해 특정 유형의 질문을 생성하도록 유도하거나, 생성되는 질문의 개수를 조절하는 등 커스터마이징이 가능함.
  - 여기서는 질문을 3가지 버전으로 다양하게 생성하되, 원래 의도를 유지하도록 하는 프롬프트를 사용함.

In [8]:
# 출력 파서: LLM 결과를 줄바꿈 기준으로 나눠서 질문 리스트로 변환
class LineListOutputParser(BaseOutputParser[List[str]]):
    """Output parser for a list of lines."""
    def parse(self, text: str) -> List[str]:
        return [line.strip() for line in text.strip().split("\n") if line.strip()]
    
QUERY_PROMPT_CUSTOM = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant.
    Your task is to generate three different versions of the given user question to retrieve relevant documents from a vector database. 
    By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of distance-based similarity search.
    Provide these alternative questions separated by newlines.
    Original question: {question}
    Alternative questions:""",
)

llm_custom_multi_query = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 멀티쿼리 생성 체인 구성
multiquery_chain_custom = QUERY_PROMPT_CUSTOM | llm_custom_multi_query | LineListOutputParser()

# 테스트 쿼리 실행 (생성된 대안 질문들 확인)
query_for_custom = "리비안의 성장 동력은 무엇인가요?"
alternative_questions = multiquery_chain_custom.invoke({"question": query_for_custom})

print(f"원본 질문: {query_for_custom}")
print("생성된 대안 질문들:")
for i, q in enumerate(alternative_questions, 1):
    print(f"{i}. {q}")

원본 질문: 리비안의 성장 동력은 무엇인가요?
생성된 대안 질문들:
1. 리비안의 성장 요인은 무엇인지 알고 싶습니다.
2. 리비안이 성장하는 데 기여하는 주요 요소는 무엇인가요?
3. 리비안의 발전을 이끄는 힘은 어떤 것들이 있나요?


`(2) Decomposition (질문 분해)`

**원리**: 복잡한 질문을 여러 개의 간단한 하위 질문으로 분해함. 각 하위 질문에 대해 검색을 수행하고, 그 결과를 종합하여 원래 질문에 대한 답을 찾음. 예를 들어 "테슬라의 역사와 주요 경쟁사는?"라는 질문을 "테슬라의 역사는?"와 "테슬라의 주요 경쟁사는?"로 나누는 식임. (LEAST-TO-MOST PROMPTING과 유사한 아이디어)

**장점**:
-   **복잡한 질문 처리 용이**: 여러 정보를 한 번에 묻는 질문에 효과적임. 각 정보 조각을 개별적으로 검색하여 정확도를 높일 수 있음.
-   **정보 누락 방지**: 각 하위 질문에 대한 답을 찾으므로, 복잡한 질문의 특정 부분을 놓치지 않고 정보를 얻을 수 있음.

**단점**:
-   **LLM 호출 비용 및 시간**: 질문을 분해하고, 각 하위 질문에 대해 처리하는 데 LLM 호출이 필요할 수 있음.
-   **분해 정확도 의존**: LLM이 질문을 잘못 분해하면, 엉뚱한 정보를 찾거나 중요한 정보를 놓칠 수 있음. 프롬프트 엔지니어링이 중요.
-   **결과 통합의 복잡성**: 분해된 질문들로부터 얻은 다양한 정보 조각들을 최종적으로 어떻게 의미있게 통합할지가 관건.

`(2) Decomposition (질문 분해)`

**원리**
- 복잡한 질문을 여러 개의 간단한 하위 질문으로 분해함. 각 하위 질문에 대해 검색을 수행하고, 그 결과를 종합하여 원래 질문에 대한 답을 찾음. 
- 예를 들어 "테슬라의 역사와 주요 경쟁사는?"라는 질문을 "테슬라의 역사는?"와 "테슬라의 주요 경쟁사는?"로 나누는 식임. (LEAST-TO-MOST PROMPTING과 유사한 아이디어)

**장점**:
-   **복잡한 질문 처리 용이**: 여러 정보를 한 번에 묻는 질문에 효과적임. 각 정보 조각을 개별적으로 검색하여 정확도를 높일 수 있음.
-   **정보 누락 방지**: 각 하위 질문에 대한 답을 찾으므로, 복잡한 질문의 특정 부분을 놓치지 않고 정보를 얻을 수 있음.

**단점**:
-   **LLM 호출 비용 및 시간**: 질문을 분해하고, 각 하위 질문에 대해 처리하는 데 LLM 호출이 필요할 수 있음.
-   **분해 정확도 의존**: LLM이 질문을 잘못 분해하면, 엉뚱한 정보를 찾거나 중요한 정보를 놓칠 수 있음. 프롬프트 엔지니어링이 중요.
-   **결과 통합의 복잡성**: 분해된 질문들로부터 얻은 다양한 정보 조각들을 최종적으로 어떻게 의미있게 통합할지가 관건.

In [9]:
DECOMPOSITION_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to decompose the given input question into multiple sub-questions. 
    The goal is to break down the input into a set of sub-problems/sub-questions that can be answered independently.

    Follow these guidelines to generate the sub-questions:
    1. Cover various aspects related to the core topic of the original question.
    2. Each sub-question should be specific, clear, and answerable independently.
    3. Ensure that the sub-questions collectively address all important aspects of the original question.
    4. Consider temporal aspects (past, present, future) where applicable.
    5. Formulate the questions in a direct and concise manner.
    6. Provide these sub-questions separated by newlines.

    [Input question] 
    {question}

    [Sub-questions]
    """,
)

llm_decomposition = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 질문 분해 체인
decomposition_chain = DECOMPOSITION_PROMPT | llm_decomposition | LineListOutputParser()

query_for_decomposition = "리비안의 역사와 주요 성장 동력은 무엇인가요?"
sub_questions = decomposition_chain.invoke({"question": query_for_decomposition})

print(f"원본 질문: {query_for_decomposition}")
print("생성된 서브 질문들:")
for i, q in enumerate(sub_questions, 1):
    print(f"{i}. {q}")

원본 질문: 리비안의 역사와 주요 성장 동력은 무엇인가요?
생성된 서브 질문들:
1. 리비안의 설립 연도와 초기 배경은 무엇인가요?
2. 리비안의 주요 제품과 기술은 무엇인가요?
3. 리비안의 성장에 기여한 주요 투자자나 파트너는 누구인가요?
4. 리비안이 직면한 주요 도전 과제는 무엇인가요?
5. 리비안의 시장 전략과 목표 고객층은 어떻게 설정되어 있나요?
6. 리비안의 미래 성장 전망은 어떻게 예상되나요?
7. 리비안의 경쟁사와 그들과의 차별점은 무엇인가요?


In [10]:
# 분해된 질문들을 사용하는 MultiQueryRetriever (사실상 같은 원리)
multi_query_decomposition_retriever = MultiQueryRetriever(
    retriever=chroma_k_retriever, # 기본 검색기 (k=2)   
    llm_chain=decomposition_chain,   
)  

retrieved_docs_decomposition = multi_query_decomposition_retriever.invoke(query_for_decomposition)

print(f"\n[Decomposition 기반 검색] 쿼리: {query_for_decomposition}")
print("검색 결과:")
unique_docs_decomposition = {doc.page_content: doc for doc in retrieved_docs_decomposition} # 내용 기반 중복 제거
for i, doc in enumerate(unique_docs_decomposition.values()):
    print(f"문서 {i+1}:")
    print(f"- 내용: {doc.page_content}")
    print(f"- 출처: {doc.metadata['source']}")
    print("-"*50)


[Decomposition 기반 검색] 쿼리: 리비안의 역사와 주요 성장 동력은 무엇인가요?
검색 결과:
문서 1:
- 내용: Rivian Automotive, Inc.는 미국의 전기 자동차 제조업체이자 자동차 기술 회사입니다. 
본사는 캘리포니아 어바인에 있습니다. Rivian은 전기 SUV와 픽업 트럭을 생산합니다.
Rivian의 첫 번째 차량은 2021년에 공개된 R1T 전기 픽업 트럭입니다.
R1S라는 SUV 모델도 있습니다. Rivian은 배송용 전기 밴도 개발하여 아마존에 공급하고 있습니다.
Rivian의 성장 동력은 혁신적인 전기차 기술과 강력한 시장 수요입니다.
회사는 2009년에 설립되었으며, 초기에는 다른 이름으로 시작했으나 2011년에 Rivian으로 변경했습니다.
- 출처: sample_data\rivian.txt
--------------------------------------------------
문서 2:
- 내용: 인공지능(AI)은 빠르게 발전하고 있는 기술 분야입니다.
머신러닝과 딥러닝은 AI의 핵심 구성 요소입니다.
최근에는 대규모 언어 모델(LLM)이 주목받고 있으며, 다양한 서비스에 활용되고 있습니다.
- 출처: sample_data\general_tech.txt
--------------------------------------------------
문서 3:
- 내용: Tesla, Inc.는 미국의 전기 자동차 및 청정 에너지 회사입니다. 
본사는 텍사스 오스틴에 있습니다. Tesla는 전기 자동차, 태양광 패널, 배터리 에너지 저장 장치를 설계하고 제조합니다.
일론 머스크 (Elon Musk)는 Tesla의 CEO이자 제품 설계자입니다. 
그는 Tesla의 장기적인 전략적 방향과 모든 제품 개발을 이끌고 있습니다.
로빈 덴홈 (Robyn Denholm)은 Tesla 이사회 의장입니다.
Tesla의 주요 차량 모델에는 Model S, Model 3, Model X, Model Y, 그리고 Cybertruck이 있습

### 4-2. Re-rank (재순위화)

- 일단 리트리버가 여러 문서를 가져왔다고 해서 그 순서가 항상 최선은 아님.
- 재순위화는 1차로 검색된 문서들을 **사용자 질문과의 관련성을 기준으로 다시 한번 정교하게 평가**해서 순서를 매기는 과정임.
- 마치 오디션에서 1차 합격자들을 모아놓고 2차 심층 면접을 보는 것과 같음.

**왜 필요함?**
- 기본적인 벡터 유사도 검색(예: 코사인 유사도)은 빠르지만, 문맥의 미묘한 차이나 질문의 복잡한 의도를 완벽히 반영하지 못할 수 있음.
- 재순위화는 더 강력한 모델(Cross-Encoder 또는 LLM)을 사용해 이 부분을 보완함.

In [11]:
# 재순위화를 위해 초기 검색 문서 수를 늘림 (예: k=5)
chroma_k_retriever_for_rerank = chroma_db_loaded.as_retriever(
    search_kwargs={"k": 5} 
)
print(f"재순위화용 기본 검색기 k값: {chroma_k_retriever_for_rerank.search_kwargs['k']}")

재순위화용 기본 검색기 k값: 5


`(1) Cross Encoder Reranker`

**원리**: Cross-Encoder 모델은 (질문, 문서) 쌍을 입력으로 받아, 질문과 문서 사이의 관련성 점수를 직접 계산함. Bi-Encoder(임베딩 모델처럼 질문과 문서를 각각 인코딩 후 유사도 계산)보다 일반적으로 더 정확한 관련성 판단이 가능함. Hugging Face의 `BAAI/bge-reranker-v2-m3` 같은 모델이 여기에 사용됨.

**장점**:
-   **높은 정확도**: 질문과 문서를 함께 고려하여 문맥적 관련성을 깊이 있게 판단하므로, 재순위화 성능이 매우 우수함.
-   **미세 조정 가능**: 특정 도메인 데이터에 파인튜닝하여 성능을 더욱 향상시킬 수 있음.

**단점**:
-   **계산 비용 높음**: 각 (질문, 문서) 쌍마다 모델을 통과시켜야 하므로, 문서 수가 많으면 계산량이 많아져 속도가 느릴 수 있음. 그래서 보통 1차 검색된 소수의 후보 문서에 대해서만 적용함.
-   **Bi-Encoder 대비 느린 속도**: Bi-Encoder는 문서 임베딩을 미리 계산해둘 수 있지만, Cross-Encoder는 검색 시점에 질문과 각 문서를 쌍으로 처리해야 함.

In [13]:
cross_encoder_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
re_ranker_cross_encoder = CrossEncoderReranker(model=cross_encoder_model, top_n=2) # 상위 2개만 선택

cross_encoder_reranker_retriever = ContextualCompressionRetriever(
    base_compressor=re_ranker_cross_encoder, 
    base_retriever=chroma_k_retriever_for_rerank, # k=5로 설정된 리트리버 사용
)

query_for_rerank = "테슬라 회장은 누구인가요?"

print(f"[CrossEncoder Reranker] 쿼리: {query_for_rerank}")
print(f"초기 검색(k=5) 후 CrossEncoder로 top_n=2 재선별")
retrieved_docs_cross_encoder = cross_encoder_reranker_retriever.invoke(query_for_rerank)


Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


[CrossEncoder Reranker] 쿼리: 테슬라 회장은 누구인가요?
초기 검색(k=5) 후 CrossEncoder로 top_n=2 재선별


In [19]:
print("검색 결과:")
for i, doc in enumerate(retrieved_docs_cross_encoder):
    # CrossEncoderReranker는 metadata에 'relevance_score'를 추가함
    print(f"- 내용: {doc.page_content}")
    print(f"- 출처: {doc.metadata['source']}")
    print("-"*50)

검색 결과:
- 내용: Tesla, Inc.는 미국의 전기 자동차 및 청정 에너지 회사입니다. 
본사는 텍사스 오스틴에 있습니다. Tesla는 전기 자동차, 태양광 패널, 배터리 에너지 저장 장치를 설계하고 제조합니다.
일론 머스크 (Elon Musk)는 Tesla의 CEO이자 제품 설계자입니다. 
그는 Tesla의 장기적인 전략적 방향과 모든 제품 개발을 이끌고 있습니다.
로빈 덴홈 (Robyn Denholm)은 Tesla 이사회 의장입니다.
Tesla의 주요 차량 모델에는 Model S, Model 3, Model X, Model Y, 그리고 Cybertruck이 있습니다.
- 출처: sample_data\tesla.txt
--------------------------------------------------
- 내용: Rivian Automotive, Inc.는 미국의 전기 자동차 제조업체이자 자동차 기술 회사입니다. 
본사는 캘리포니아 어바인에 있습니다. Rivian은 전기 SUV와 픽업 트럭을 생산합니다.
Rivian의 첫 번째 차량은 2021년에 공개된 R1T 전기 픽업 트럭입니다.
R1S라는 SUV 모델도 있습니다. Rivian은 배송용 전기 밴도 개발하여 아마존에 공급하고 있습니다.
Rivian의 성장 동력은 혁신적인 전기차 기술과 강력한 시장 수요입니다.
회사는 2009년에 설립되었으며, 초기에는 다른 이름으로 시작했으나 2011년에 Rivian으로 변경했습니다.
- 출처: sample_data\rivian.txt
--------------------------------------------------


`(2) LLM Reranker (LLMListwiseRerank)`

**원리**: 
- LLM을 사용하여 1차 검색된 문서 목록 전체를 한 번에 보고, 질문과의 관련성에 따라 순서를 조정함. 
- LLM에게 "이 질문이 있고, 여기 문서 목록이 있는데, 질문과 가장 관련 높은 순서대로 다시 정렬해줘"라고 요청하는 방식임. `LLMListwiseRerank`가 이런 역할을 함.

**장점**:
-   **정교한 문맥 이해**: LLM의 강력한 언어 이해 능력을 활용하여 문서 간의 미묘한 차이나 질문의 복잡한 의도를 파악하여 순서를 정할 수 있음.
-   **유연성**: 프롬프트를 조정하여 재순위화 기준을 다양하게 설정할 수 있음 (예: 최신 정보 우선, 특정 키워드 포함 문서 우선 등).

**단점**:
-   **LLM 호출 비용 및 시간**: 문서 목록을 LLM에 전달하고 처리하는 데 비용과 시간이 소요됨. 특히 문서의 양이 많거나 내용이 길면 컨텍스트 윈도우 제한 및 비용 문제가 커질 수 있음.
-   **프롬프트 민감성**: LLM의 성능은 프롬프트에 크게 의존하므로, 효과적인 재순위화를 위해서는 프롬프트 엔지니어링이 중요할 수 있음.
-   **일관성 문제**: LLM의 특성상 동일 입력에도 약간 다른 결과를 낼 수 있음 (temperature 조절로 완화 가능).

In [20]:
llm_for_rerank = ChatOpenAI(model="gpt-4o-mini", temperature=0)

re_ranker_llm = LLMListwiseRerank.from_llm(llm_for_rerank, top_n=2) # LLM이 재정렬 후 상위 2개 선택
llm_reranker_retriever = ContextualCompressionRetriever(
    base_compressor=re_ranker_llm, 
    base_retriever=chroma_k_retriever_for_rerank, # k=5로 설정된 리트리버 사용
)

query_for_llm_rerank = "테슬라 회장은 누구인가요?"

print(f"[LLM Reranker] 쿼리: {query_for_llm_rerank}")
print(f"초기 검색(k=5) 후 LLMListwiseRerank로 top_n=2 재선별")
retrieved_docs_llm_rerank = llm_reranker_retriever.invoke(query_for_llm_rerank)

print("검색 결과:")
for i, doc in enumerate(retrieved_docs_llm_rerank):
    # LLMListwiseRerank는 기본적으로 점수를 metadata에 추가하지 않음 (필요시 커스텀 필요)
    print(f"문서 {i+1}:") 
    print(f"- 내용: {doc.page_content}")
    print(f"- 출처: {doc.metadata['source']}")
    print("-"*50)

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


[LLM Reranker] 쿼리: 테슬라 회장은 누구인가요?
초기 검색(k=5) 후 LLMListwiseRerank로 top_n=2 재선별
검색 결과:
문서 1:
- 내용: Tesla, Inc.는 미국의 전기 자동차 및 청정 에너지 회사입니다. 
본사는 텍사스 오스틴에 있습니다. Tesla는 전기 자동차, 태양광 패널, 배터리 에너지 저장 장치를 설계하고 제조합니다.
일론 머스크 (Elon Musk)는 Tesla의 CEO이자 제품 설계자입니다. 
그는 Tesla의 장기적인 전략적 방향과 모든 제품 개발을 이끌고 있습니다.
로빈 덴홈 (Robyn Denholm)은 Tesla 이사회 의장입니다.
Tesla의 주요 차량 모델에는 Model S, Model 3, Model X, Model Y, 그리고 Cybertruck이 있습니다.
- 출처: sample_data\tesla.txt
--------------------------------------------------
문서 2:
- 내용: Rivian Automotive, Inc.는 미국의 전기 자동차 제조업체이자 자동차 기술 회사입니다. 
본사는 캘리포니아 어바인에 있습니다. Rivian은 전기 SUV와 픽업 트럭을 생산합니다.
Rivian의 첫 번째 차량은 2021년에 공개된 R1T 전기 픽업 트럭입니다.
R1S라는 SUV 모델도 있습니다. Rivian은 배송용 전기 밴도 개발하여 아마존에 공급하고 있습니다.
Rivian의 성장 동력은 혁신적인 전기차 기술과 강력한 시장 수요입니다.
회사는 2009년에 설립되었으며, 초기에는 다른 이름으로 시작했으나 2011년에 Rivian으로 변경했습니다.
- 출처: sample_data\rivian.txt
--------------------------------------------------


### 4-3. Contextual compression (맥락적 압축)

**원리**:
- 검색된 전체 문서를 그대로 사용하는 대신, **주어진 질문(쿼리)의 맥락을 고려하여 문서 내용을 압축**하거나 필터링하는 기법임.
- 목표는 최종 LLM에게 전달되는 컨텍스트의 양을 줄여서, LLM 호출 비용을 절감하고, 관련 없는 정보로 인한 '노이즈'를 줄여 답변의 질을 높이는 것임.

**구성 요소**:
1.  **기본 검색기 (Base Retriever)**: 1차적으로 문서를 검색해옴.
2.  **문서 압축기 (Document Compressor)**: 검색된 문서를 쿼리 맥락에 맞게 압축/필터링함.

재순위화(Re-rank)도 일종의 문서 압축기로 볼 수 있음 (`top_n`으로 문서 수를 줄이니까).

`(1) LLMChainFilter`

**원리**: 
- LLM을 사용하여 1차 검색된 각 문서가 질문과 관련이 있는지 판단하고, 관련 없는 문서는 걸러냄 (필터링). 
- 문서 내용을 변경하거나 요약하지는 않고, 통째로 남기거나 버리거나 둘 중 하나임.

**장점**:
-   **관련성 높은 문서 선별**: LLM의 이해력을 바탕으로 질문과 관련성이 낮은 문서를 효과적으로 제거하여 노이즈를 줄임.
-   **구현 용이성**: LangChain에서 쉽게 적용 가능.

**단점**:
-   **LLM 호출 비용**: 각 문서마다 관련성 판단을 위해 LLM을 호출할 수 있어 비용과 시간이 소요됨 (구현에 따라 한 번의 호출로 여러 문서를 판단하게 할 수도 있음).
-   **정보 손실 가능성**: LLM이 너무 엄격하게 필터링하거나 판단을 잘못하면, 유용한 정보가 담긴 문서가 실수로 제외될 수 있음.
-   **문서 내용 변경 없음**: 문서 자체를 압축하는 것은 아니므로, 여전히 긴 문서가 전달될 수 있음.

In [21]:
llm_for_filter = ChatOpenAI(model="gpt-4o-mini", temperature=0)
context_filter = LLMChainFilter.from_llm(llm_for_filter)

llm_filter_compression_retriever = ContextualCompressionRetriever(
    base_compressor=context_filter,             
    base_retriever=chroma_db_loaded.as_retriever(search_kwargs={"k": 3}), # 필터링할 후보 3개 가져오기
)

query_for_filter = "테슬라 회장은 누구인가요?"

print(f"[LLMChainFilter] 쿼리: {query_for_filter}")
print(f"초기 검색(k=3) 후 LLMChainFilter로 관련 문서만 선별")
compressed_docs_filter = llm_filter_compression_retriever.invoke(query_for_filter)

print("검색 결과 (필터링 후 남은 문서 수:", len(compressed_docs_filter),"):")
for i, doc in enumerate(compressed_docs_filter):
    print(f"문서 {i+1}:")
    print(f"- 내용: {doc.page_content}")
    print(f"- 출처: {doc.metadata['source']}")
    print("-"*50)

[LLMChainFilter] 쿼리: 테슬라 회장은 누구인가요?
초기 검색(k=3) 후 LLMChainFilter로 관련 문서만 선별
검색 결과 (필터링 후 남은 문서 수: 1 ):
문서 1:
- 내용: Tesla, Inc.는 미국의 전기 자동차 및 청정 에너지 회사입니다. 
본사는 텍사스 오스틴에 있습니다. Tesla는 전기 자동차, 태양광 패널, 배터리 에너지 저장 장치를 설계하고 제조합니다.
일론 머스크 (Elon Musk)는 Tesla의 CEO이자 제품 설계자입니다. 
그는 Tesla의 장기적인 전략적 방향과 모든 제품 개발을 이끌고 있습니다.
로빈 덴홈 (Robyn Denholm)은 Tesla 이사회 의장입니다.
Tesla의 주요 차량 모델에는 Model S, Model 3, Model X, Model Y, 그리고 Cybertruck이 있습니다.
- 출처: sample_data\tesla.txt
--------------------------------------------------


`(2) LLMChainExtractor`

**원리**: L
- LM을 사용하여 1차 검색된 각 문서에서 **질문과 관련된 내용만 추출하여 요약**함. 
- 문서 전체를 반환하는 대신, 질문에 대한 답변과 직접적으로 관련된 문장이나 구문만 뽑아내는 방식임.

**장점**:
-   **높은 관련성**: 질문과 직접 관련된 핵심 정보만 추출하므로, 최종 LLM에 전달되는 컨텍스트의 질이 매우 높아짐.
-   **컨텍스트 크기 감소**: 불필요한 부분을 제거하고 핵심만 남기므로, LLM의 컨텍스트 윈도우를 효율적으로 사용하고 처리 속도를 높일 수 있음.

**단점**:
-   **LLM 호출 비용**: 각 문서의 내용을 요약/추출하기 위해 LLM을 호출하므로 비용과 시간이 많이 소요될 수 있음.
-   **정보 손실 위험**: LLM이 중요한 정보를 누락하거나 잘못 요약할 경우, 답변의 질이 저하될 수 있음. 추출 과정에서 문맥이 왜곡될 수도 있음.
-   **프롬프트 중요성**: 추출의 정확도는 LLM에게 주어지는 프롬프트에 크게 좌우됨.

In [22]:
llm_for_extractor = ChatOpenAI(model="gpt-4o-mini", temperature=0)
compressor_extractor = LLMChainExtractor.from_llm(llm_for_extractor)

# 여기서는 앞에서 만든 CrossEncoder 재순위화 결과를 기본 리트리버로 사용해봄 (선택 사항)
# 즉, 1차 검색(k=5) -> CrossEncoder로 재순위화(top_n=2) -> LLMChainExtractor로 내용 추출
llm_extractor_compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor_extractor,                                    
    base_retriever=cross_encoder_reranker_retriever, # 이미 재순위화된 결과를 사용 (top_n=2 였음)
)

query_for_extractor = "테슬라 회장은 누구인가요?"

print(f"[LLMChainExtractor] 쿼리: {query_for_extractor}")
print(f"CrossEncoder Reranker 결과(2개 문서)에 대해 LLMChainExtractor로 내용 추출")
compressed_docs_extractor = llm_extractor_compression_retriever.invoke(query_for_extractor)

print("검색 결과 (추출/요약 후 남은 문서 수:", len(compressed_docs_extractor),"):")
for i, doc in enumerate(compressed_docs_extractor):
    print(f"문서 {i+1}:")
    print(f"- 추출된 내용: {doc.page_content}") # 내용이 질문에 맞게 압축/요약됨
    print(f"- 원본 출처: {doc.metadata['source']}")
    print("-"*50)

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


[LLMChainExtractor] 쿼리: 테슬라 회장은 누구인가요?
CrossEncoder Reranker 결과(2개 문서)에 대해 LLMChainExtractor로 내용 추출
검색 결과 (추출/요약 후 남은 문서 수: 1 ):
문서 1:
- 추출된 내용: 일론 머스크 (Elon Musk)는 Tesla의 CEO이자 제품 설계자입니다. 
로빈 덴홈 (Robyn Denholm)은 Tesla 이사회 의장입니다.
- 원본 출처: sample_data\tesla.txt
--------------------------------------------------


`(3) EmbeddingsFilter`

**원리**: 
- 1차 검색된 문서들의 임베딩과 원본 질문(쿼리)의 임베딩 사이의 유사도를 다시 계산함. 
- 이 유사도가 설정된 임계값(similarity_threshold)보다 낮은 문서들은 필터링하여 제거함. 
- LLM을 사용하지 않는 방식임.

**장점**:
-   **속도 빠름 & 비용 저렴**: LLM 호출 없이 임베딩 유사도 계산만으로 필터링하므로 매우 빠르고 비용 효율적임.
-   **간단한 구현**: 설정이 간단하고 직관적임.

**단점**:
-   **임계값 설정의 어려움**: 적절한 유사도 임계값을 찾는 것이 중요함. 너무 높으면 유용한 문서가 걸러지고, 너무 낮으면 필터링 효과가 미미할 수 있음. 데이터와 사용 사례에 따라 조정 필요.
-   **의미론적 한계**: 단순 유사도 기반이므로, 표면적으로는 유사해 보이지만 실제로는 관련 없거나, 반대로 표현은 다르지만 의미상 중요한 문서를 놓칠 수 있음. (LLM 필터보다는 정교함이 떨어짐)

In [23]:
# embeddings_model은 위에서 정의한 BAAI/bge-m3 사용
embeddings_filter = EmbeddingsFilter(embeddings=embeddings_model, similarity_threshold=0.6) # 임계값 0.6 (조정 가능, BGE-M3는 점수 범위가 넓으므로 테스트 필요)

# 여기서는 CrossEncoder 재순위화 결과를 기본 리트리버로 사용
embed_filter_compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,                             
    base_retriever=cross_encoder_reranker_retriever, # 이미 재순위화된 결과를 사용 (top_n=2 였음)
)

query_for_embed_filter = "테슬라 회장은 누구인가요?"

print(f"[EmbeddingsFilter] 쿼리: {query_for_embed_filter}")
print(f"CrossEncoder Reranker 결과(2개 문서)에 대해 EmbeddingsFilter (threshold=0.6) 적용")
compressed_docs_embed_filter = embed_filter_compression_retriever.invoke(query_for_embed_filter)

print("검색 결과 (임베딩 필터링 후 남은 문서 수:", len(compressed_docs_embed_filter),"):")
for i, doc in enumerate(compressed_docs_embed_filter):
    print(f"문서 {i+1}:")
    print(f"- 내용: {doc.page_content}")
    print(f"- 출처: {doc.metadata['source']}")
    print("-"*50)

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


[EmbeddingsFilter] 쿼리: 테슬라 회장은 누구인가요?
CrossEncoder Reranker 결과(2개 문서)에 대해 EmbeddingsFilter (threshold=0.6) 적용
검색 결과 (임베딩 필터링 후 남은 문서 수: 1 ):
문서 1:
- 내용: Tesla, Inc.는 미국의 전기 자동차 및 청정 에너지 회사입니다. 
본사는 텍사스 오스틴에 있습니다. Tesla는 전기 자동차, 태양광 패널, 배터리 에너지 저장 장치를 설계하고 제조합니다.
일론 머스크 (Elon Musk)는 Tesla의 CEO이자 제품 설계자입니다. 
그는 Tesla의 장기적인 전략적 방향과 모든 제품 개발을 이끌고 있습니다.
로빈 덴홈 (Robyn Denholm)은 Tesla 이사회 의장입니다.
Tesla의 주요 차량 모델에는 Model S, Model 3, Model X, Model Y, 그리고 Cybertruck이 있습니다.
- 출처: sample_data\tesla.txt
--------------------------------------------------


`(4) DocumentCompressorPipeline`

**원리**: 
- 여러 문서 변환기(Document Transformers)와 압축기(Compressors)를 파이프라인 형태로 순차적으로 결합하여 사용하는 방식임. 
- 예를 들어, 먼저 중복 문서를 제거하고(`EmbeddingsRedundantFilter`), 그 다음 임베딩 기반으로 관련 없는 문서를 필터링하고(`EmbeddingsFilter`), 마지막으로 LLM으로 재순위화(`LLMListwiseRerank`)하는 식으로 구성할 수 있음.

**장점**:
-   **고도의 맞춤화 가능**: 다양한 변환기와 압축기를 조합하여 매우 정교하고 복잡한 문서 처리 로직을 구성할 수 있음. 각 단계의 강점을 활용 가능.
-   **단계별 처리**: 각 단계를 거치면서 문서의 질을 점진적으로 향상시킬 수 있음.

**단점**:
-   **복잡성 증가**: 여러 컴포넌트를 조합하므로 전체 시스템의 복잡도가 높아지고, 각 단계의 설정 및 튜닝이 필요함.
-   **오류 전파 가능성**: 파이프라인의 초기 단계에서 오류가 발생하거나 잘못된 처리가 이루어지면, 후속 단계에도 영향을 미칠 수 있음.
-   **성능 병목**: 파이프라인 중 특정 단계가 느리면 전체 처리 속도에 영향을 줄 수 있음.

In [24]:
# 1. 중복 문서 제거 필터 (임베딩 기반)
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings_model)

# 2. 관련성 낮은 문서 필터 (임베딩 기반)
relevant_filter = EmbeddingsFilter(embeddings=embeddings_model, similarity_threshold=0.6) # 임계값 조정 가능

# 3. LLM 기반 재순위화
llm_for_pipeline_rerank = ChatOpenAI(model="gpt-4o-mini", temperature=0)
re_ranker_pipeline = LLMListwiseRerank.from_llm(llm_for_pipeline_rerank, top_n=1) # 최종적으로 1개만 선택

# 파이프라인 압축기 구성
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[redundant_filter, relevant_filter, re_ranker_pipeline]
)

pipeline_compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, 
    base_retriever=chroma_db_loaded.as_retriever(search_kwargs={"k": 5}) # 파이프라인 처리를 위해 초기 5개 문서 검색
)

query_for_pipeline = "테슬라 회장은 누구인가요?"

print(f"[DocumentCompressorPipeline] 쿼리: {query_for_pipeline}")
print(f"초기 검색(k=5) -> 중복제거 -> 관련성필터(thresh=0.6) -> LLM재순위(top_n=1)")
compressed_docs_pipeline = pipeline_compression_retriever.invoke(query_for_pipeline)

print("검색 결과 (파이프라인 처리 후 남은 문서 수:", len(compressed_docs_pipeline),"):")
for i, doc in enumerate(compressed_docs_pipeline):
    print(f"문서 {i+1}:")
    print(f"- 내용: {doc.page_content}")
    print(f"- 출처: {doc.metadata['source']}")
    print("-"*50)

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


[DocumentCompressorPipeline] 쿼리: 테슬라 회장은 누구인가요?
초기 검색(k=5) -> 중복제거 -> 관련성필터(thresh=0.6) -> LLM재순위(top_n=1)
검색 결과 (파이프라인 처리 후 남은 문서 수: 1 ):
문서 1:
- 내용: Tesla, Inc.는 미국의 전기 자동차 및 청정 에너지 회사입니다. 
본사는 텍사스 오스틴에 있습니다. Tesla는 전기 자동차, 태양광 패널, 배터리 에너지 저장 장치를 설계하고 제조합니다.
일론 머스크 (Elon Musk)는 Tesla의 CEO이자 제품 설계자입니다. 
그는 Tesla의 장기적인 전략적 방향과 모든 제품 개발을 이끌고 있습니다.
로빈 덴홈 (Robyn Denholm)은 Tesla 이사회 의장입니다.
Tesla의 주요 차량 모델에는 Model S, Model 3, Model X, Model Y, 그리고 Cybertruck이 있습니다.
- 출처: sample_data\tesla.txt
--------------------------------------------------


## 5. 답변 생성 (RAG 파이프라인)

지금까지 다양한 고급 검색 기법들을 통해 질문과 관련된 문서를 효과적으로 찾아왔음.
이제 이 문서들을 컨텍스트(Context)로 활용하여 LLM에게 최종 답변을 생성하도록 요청할 차례임.
이것이 바로 RAG(Retrieval Augmented Generation)의 핵심임!

먼저, 아무런 컨텍스트 없이 LLM이 사전 학습된 지식만으로 답변하는 경우를 보겠음.

In [25]:
# Context 없이 사전학습된 상태에서 답변 생성
llm_answer_gen = ChatOpenAI(model="gpt-4o-mini")
question_final = "테슬라 회장은 누구인가요?"

print(f"질문: {question_final}")
response_without_context = llm_answer_gen.invoke(question_final)
print(f"LLM 자체 답변 (컨텍스트 X):\n{response_without_context.content}")

질문: 테슬라 회장은 누구인가요?
LLM 자체 답변 (컨텍스트 X):
테슬라의 회장은 일론 머스크(Elon Musk)입니다. 그는 2004년에 테슬라에 합류하여 이후 CEO로서 회사를 이끌고 있습니다. 일론 머스크는 전기차, 재생 가능한 에너지, 우주 탐사 등 다양한 분야에서 혁신적인 기업들을 운영하고 있는 기업가입니다.


이제 위에서 만들었던 `pipeline_compression_retriever` (중복제거 + 관련성필터 + LLM재순위화)를 사용해서 찾아온 문서를 컨텍스트로 제공하고, RAG 체인을 구성하여 답변을 생성해보겠음.

**RAG 체인 구성 요소**:
1.  **Retriever**: `pipeline_compression_retriever`가 질문에 맞는 문서를 찾아옴.
2.  **`format_docs`**: 찾아온 문서(Document 객체 리스트)를 LLM 프롬프트에 넣기 좋은 문자열 형태로 변환함.
3.  **`RunnablePassthrough`**: 사용자 질문을 그대로 다음 단계로 전달함.
4.  **Prompt**: LLM에게 컨텍스트와 질문을 함께 제공하며 어떻게 답변해야 할지 지시하는 템플릿.
5.  **LLM**: `ChatOpenAI` 모델이 프롬프트를 기반으로 답변을 생성함.
6.  **`StrOutputParser`**: LLM의 출력(채팅 메시지 객체)에서 실제 텍스트 답변만 추출함.

In [27]:
template_rag = """당신은 질문에 대해 주어진 컨텍스트를 기반으로 답변하는 AI 어시스턴트입니다.
컨텍스트에 질문과 관련된 내용이 있다면, 해당 내용을 근거로 답변해주세요.
만약 컨텍스트에서 답변을 찾을 수 없다면, "제공된 정보만으로는 답변을 찾을 수 없습니다."라고 솔직하게 답변해주세요.
답변은 한국어로 해주세요.

[컨텍스트]
{context}

[질문]
{question}

[답변]
"""

prompt_rag = ChatPromptTemplate.from_template(template_rag)

def format_docs(docs):
    # docs가 비어있을 경우 처리
    if not docs:
        return "제공된 컨텍스트가 없습니다."
    return "\n\n".join([f"[문서 출처: {doc.metadata.get('source', '알 수 없음')}]\n{doc.page_content}" for doc in docs])

# 가장 마지막에 테스트했던 pipeline_compression_retriever를 사용
final_retriever = pipeline_compression_retriever 

rag_chain = (
    {"context": final_retriever | format_docs, "question": RunnablePassthrough()} 
    | prompt_rag
    | llm_answer_gen # 위에서 정의한 gpt-4o-mini 모델
    | StrOutputParser()
)

response_with_context = rag_chain.invoke(question_final)
print(f"질문: {question_final}")
print(f"RAG 답변 (컨텍스트 O):\n{response_with_context}")

Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


질문: 테슬라 회장은 누구인가요?
RAG 답변 (컨텍스트 O):
테슬라의 이사회 의장은 로빈 덴홈 (Robyn Denholm)입니다.


## 6. 정리 및 결론

이 노트북에서는 LangChain에서 제공하는 다양한 고급 검색 기법들을 살펴봤음.

-   **쿼리 확장 (MultiQuery, Decomposition)**: 사용자의 질문을 여러 각도에서 해석하거나 분해하여 검색 범위를 넓히고 재현율을 높임.
    -   👍 장점: 더 많은 관련 문서를 찾을 수 있음.
    -   👎 단점: LLM 호출로 인한 비용/시간, 관련 없는 문서 증가 가능성.
-   **재순위화 (CrossEncoder, LLM Reranker)**: 1차 검색된 문서들을 더 정교한 모델로 다시 평가하여 순위를 조정, 정확도를 높임.
    -   👍 장점: 검색 결과의 질적 향상, 미묘한 문맥 파악.
    -   👎 단점: 추가 계산/LLM 호출로 인한 비용/시간.
-   **맥락적 압축 (LLMChainFilter, LLMChainExtractor, EmbeddingsFilter)**: 검색된 문서에서 질문과 관련된 부분만 필터링하거나 추출하여 LLM에 전달되는 컨텍스트의 효율성을 높임.
    -   👍 장점: LLM 컨텍스트 최적화, 노이즈 감소, 비용 절감 가능성.
    -   👎 단점: LLM 호출 비용(LLM 기반 압축기), 정보 손실 위험, 임계값 설정(임베딩 필터).
-   **DocumentCompressorPipeline**: 여러 압축/변환 단계를 조합하여 고도로 맞춤화된 검색 파이프라인 구축.
    -   👍 장점: 유연성, 각 단계의 강점 활용.
    -   👎 단점: 복잡성 증가, 오류 전파 가능성.

**핵심은?**
어떤 기법이 항상 최고라고 말할 순 없음. **데이터의 특성, 사용자의 요구사항, 시스템의 성능 목표(정확도, 속도, 비용 등)를 종합적으로 고려**해서 적절한 기법을 선택하고 조합하는 것이 중요함.

이런 고급 기법들을 잘 활용하면 RAG 시스템의 성능을 한층 끌어올릴 수 있을 거임! 👍